In [2]:
import sys
lib = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\python_playground\classifying-response-to-immunotherapy\utilities\droplet_dataset'
lib2 = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\python_playground\classifying-response-to-immunotherapy\utilities'
lib3 = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\python_playground\classifying-response-to-immunotherapy\data_analysis'
lib4 = r'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\python_playground\classifying-response-to-immunotherapy'
sys.path.append(lib)
sys.path.append(lib2)
sys.path.append(lib3)
sys.path.append(lib4)

import os
from os.path import join
import sklearn
from droplet_dataset import *
from utilities import *
from matplotlib import pyplot
import numpy as np
import scipy
import pickle
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import random
from scipy.stats import pearsonr
from matplotlib.pyplot import figure
sklearn.__version__
%matplotlib notebook


In [4]:
clinical_table = pd.read_excel(r'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\Data\tables\edited_unprotected_Melanoma_clinical_data.xlsx')

In [5]:
clinical_table.head(2)

,#,Patient id,MGH id,Date collected,Sex (F/M),Age (relevant to time of sample),Melanoma type,Primary=1; Metastasis=0,Genotype,Therapy(ies) prior to biopsy,Therapy after biopsy,Clinical response,Status (Alive=1/Dead=0),Biopsy site,"Biopsy type (pre, post)",Overall survival (days),Comments,Cell line (Y; N),TILs (Y; N)
0,1,M97,1256,2019-10-11,M,79.0,Cutaneous,0,TP63; DDR2; ATM; MAP2K1; HNF1A; TERT,NaN,PD1 (started on 3/31/20),R,1.0,Left Axillary LN,Pre,NaN,NaN,NaN,NaN
1,2,M98,509,2019-10-11,M,50.0,Cutaneous,0,NaN,"aPD1 (progressed on 7/29/19)- 18 cycles, start...",NA (surgical management),R (had a response in 2016-2019),1.0,Right inguinal LN- progressing lesion,Post,NaN,1126 days- PFS,NaN,NaN


RECIST criteria: 
* complete response (CR) 
* partial response (PR) for responders,
* stable disease (SD)
* progressive disease (PD) for nonresponders (Eisenhauer et al., 2009).

However, to relate molecular and cellular variables with responses of individual lesions to therapy, we classified each of the tumor samples based on radiologic assessments into:
* NR - progression/non-responder (n=31, including SD/PD samples)
* R - regression/responder (n=17, including CR/PR samples).

In [6]:
# clinical_table['Clinical response'].value_counts()

#### Taks the first 46 samples and fill nan with '?'

In [7]:
current_sample_df = clinical_table[clinical_table['Patient id'].apply(lambda x: int(x[1:])<=146)]
# current_sample_df.loc[current_sample_df['Clinical response'].isnull(), ['Clinical response']] = '??'
current_sample_df.loc[:, 'Clinical response'] = current_sample_df['Clinical response'].fillna('??')

C:\Users\KerenYlab\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


#### Maps labels from excel to CRITERIA labels

In [8]:
#### Remission or No Evidence of Disease (NED) · Complete Remission (CR) 

label_translation = {'NR': 'NR',
'NR (PD)': 'NR',
'PD': 'NR',
'??': 'not in use for now',
'R': 'R',
'R (CR)': 'R',
'NR (tumor grew after Adj PD1)': 'NR',
'SD': 'NR',
'PD on pembro; PR for enco+bini': 'R',
'NR- PD bone (R-viscerally)': 'NR',
'R except for the brain': 'R',
'NED- 10/5/20': 'R',
'NED as fo 9/8/20': 'R',
'PD for pembro but R for ipi+nivo': 'R',
'R (had a response in 2016-2019)': 'R',
'R (NED)': 'R',
'R for IPI+nivo': 'R',
'PD (on 7/8/20)': 'NR',
'NED': 'R',
'PD ': 'NR',
'NR ': 'NR',
'R (responded to Nivo)': 'R',
'NR (SD)': 'NR',
'NA- just started Tx': 'not in use for now'}

In [19]:
current_sample_df


0     [M106, M107, M108, M111, M127, M128]
1      [M99, M102, M105, M141, M145, M146]
2     [M109, M110, M115, M121, M129, M144]
3           [M116, M118, M120, M122, M133]
4                        [M97, M124, M135]
5                             [M126, M138]
6                                   [M132]
7                                   [M112]
8                                   [M114]
9                                   [M143]
10                                  [M139]
11                                  [M131]
12                                  [M103]
13                                  [M136]
14                                  [M100]
15                                  [M104]
16                                  [M123]
17                                  [M140]
18                                  [M101]
19                                   [M98]
20                                  [M137]
21                                  [M134]
22                                  [M125]
23         

In [20]:
df = current_sample_df['Clinical response'].value_counts().to_frame()
df = df.reset_index().rename(columns={'index': 'Clinical response', 'Clinical response': 'freq'})
df['response label'] = df['Clinical response'].apply(lambda x: label_translation[x])
df['samples'] = df['Clinical response'].apply(lambda x: current_sample_df[current_sample_df['Clinical response']==x]['Patient id'].values)
# ii = current_sample_df['Clinical response'].value_counts().index[3]
# label_translation[ii]
df

,Clinical response,freq,response label,samples
0,NR,6,NR,"[M106, M107, M108, M111, M127, M128]"
1,NR (PD),6,NR,"[M99, M102, M105, M141, M145, M146]"
2,??,6,not in use for now,"[M109, M110, M115, M121, M129, M144]"
3,PD,5,NR,"[M116, M118, M120, M122, M133]"
4,R,3,R,"[M97, M124, M135]"
5,R (CR),2,R,"[M126, M138]"
6,SD,1,NR,[M132]
7,NED as fo 9/8/20,1,R,[M112]
8,NR,1,NR,[M114]
9,NA- just started Tx,1,not in use for now,[M143]


In [21]:
df.to_excel(r'C:\Users\KerenYlab\Desktop\Technion studies\Keren laboratory\milestones\milestone 8 - 26.6.21\labels.xlsx', index=False)

In [22]:
df.groupby('response label').agg(['sum'])

,Clinical response,freq
,sum,sum
response label,,
NR,NRNR (PD)PDSDNR PD NR (SD)NR- PD bone (R-visce...,24
R,RR (CR)NED as fo 9/8/20R (responded to Nivo)R ...,15
not in use for now,??NA- just started Tx,7
